In [4]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
from wordcloud import WordCloud
from time import time

from konlpy import tag
from konlpy.corpus import kolaw
# from konlpy.utils import csvwrite, pprint

from nltk.tokenize import sent_tokenize

from collections import Counter


import warnings
warnings.filterwarnings('ignore')

In [9]:
Lyrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/Lyrics_v2.csv')

In [10]:
# 행복, 희망 => 1(긍정), 슬픔, 분노 => 0(부정) 으로 이진분류

def rating_to_label(label):
    if label == 0 or label == 3:
        return 1
    else:
        return 0
    
Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]

Lyrics_pos

,Lyrics,label,y
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0,1
1,라라랄라 라라랄라 이어 이어 네가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...,0,1
2,자꾸 마주치는 네 눈빛에 왠지 어색해 대체 왜 이래 고개를 숙여봐도 너의 발끝에 맘...,0,1
3,어깨동무까진 괜찮아 그 치마 손잡는 건 조금 위험해 잘 자란 메세 진 괜찮아그래도 ...,0,1
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 내 모습을 한번 ...,0,1
...,...,...,...
145,증명해 증명해 증명해 또 걔네들은 왕 이래 년째 믿고 가는 나의 촉 술 한 잔이 ...,3,1
146,들어오라 해 눈을 깔아 왜 비밀로 할 게네 부정과 죄어 전부 드러내 자릴 넘보네 ...,3,1
147,넌 대체 왜 그래난 전혀 이해 못 해 그냥 정리하란 말을 이해 못 해 넌 못 버텨...,3,1
148,이 공간 속에 비밀 가까움은 가면 같은 우린 만질 수 없는 걸 믿지 너도 알지 우...,3,1


In [26]:
pos_word = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/pos_pol_word.txt', index_col = 0)
pos_word['y'] = 1
neg_word = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/neg_pol_word.txt', index_col = 0)
neg_word['y'] = 0
knu = pd.concat([pos_word,neg_word], axis = 0).reset_index()


,index,y
0,가까이 사귀어,1
1,가까이하다,1
2,가다듬어,1
3,가다듬어 수습하는,1
4,가다듬어 수습하다,1
...,...,...
14664,비리를,0
14665,의혹,0
14666,내팽개치다,0
14667,횡령,0


In [13]:
knu = knu.to_string()

In [14]:
knu

'                                 index  y\n0                              가까이 사귀어  1\n1                                가까이하다  1\n2                                 가다듬어  1\n3                            가다듬어 수습하는  1\n4                            가다듬어 수습하다  1\n5                             가다듬어 정하다  1\n6                                  가당히  1\n7                                 가뜬하게  1\n8                            가라앉혀 바로잡다  1\n9                                 가르치며  1\n10                                 가르침  1\n11                            가벼운 마음으로  1\n12                              가볍게 웃다  1\n13                                 가볍고  1\n14                            가볍고 보드랍게  1\n15                                가분가분  1\n16                                 가분히  1\n17                                가뿐가뿐  1\n18                              가뿐가뿐하다  1\n19                               가뿐가뿐히  1\n20                                가뿐하게  1\n21                                가뿐하다  1\n22        

In [47]:
pos = []
count_pos = 0
count_neg = 0
data = Lyrics_pos['Lyrics'].to_string()

# for i in range(len(data)):
#   if 

In [48]:
len(data)

5699

In [39]:
Lyrics_pos['Lyrics'].to_string() == knu['index']

0        False
1        False
2        False
3        False
4        False
         ...  
14664    False
14665    False
14666    False
14667    False
14668    False
Name: index, Length: 14669, dtype: bool

In [35]:
Lyrics_pos['Lyrics'].to_string()

'0      아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...\n1      라라랄라 라라랄라 이어 이어 네가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...\n2      자꾸 마주치는 네 눈빛에 왠지 어색해 대체 왜 이래 고개를 숙여봐도 너의 발끝에 맘...\n3      어깨동무까진 괜찮아 그 치마 손잡는 건 조금 위험해 잘 자란 메세 진 괜찮아그래도 ...\n4      햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 내 모습을 한번 ...\n5      세상에서 가장 행복한 말 그래서 더욱 듣고픈 말 고마운 말 날 살리는 말 사랑해 그...\n6      사랑 그게 뭔지 아직도 궁금해 생각할수록 답이 멀어져 만 가는 것 같아 잘 모르겠어...\n7       널 바래다주는 길 내내 내가 변했다고 말하지 널 생각하지 않는다고 너는 투덜대지 ...\n8       만남이란 상상만으로도 눈부신 낭만의 시작한 잔의 진한 설렘에 향기가 온몸으로 퍼지...\n9      장난스러운 너의 속삭임 내 마음을 알아차린 듯 신기루 같은 사랑의 유혹 너에게 나를...\n10     뭔가 기분 좋은 일이 생길 것만 같은 기분이야 요즘 따라 년 만에 사람들이 나를 알...\n11      코코 노론 카타치카와라나이 코로 마마 도시 테 모 이이 나니 모이 라나이 아시타니...\n12     혼자라고 느낄 땐 이 멜로디를 기억해봐라 랄랄라 랄랄라 라 랄랄라라 아 언젠가 이 ...\n13     오늘같이 싱그러운 날 길거리 차도 별로 다니지 않아 문득 지나가다 거울을 보면 오늘...\n14     사랑에 빠진 순간 너만 보이고 심장은 두근거려 가슴에 폭 안겨주긴 길 걷다 뽀뽀해주...\n15     눈꽃이 내린 버스 정류장에 날 데려다주다 꼭 내 손을 잡는 너 때문에나 놀랐어 아주...\n16      이건 사랑이야 어쩌면 미쳤을지도 몰라 널 내 아기라고 불러도 돼         달...\n17     언제였던 건